# Azure Key Vault Setup

Configure the Key Encryption Key (KEK) in Azure Key Vault.

In [0]:
dbutils.widgets.text('key_vault_url', '')
dbutils.widgets.text('key_name', 'uc-demo-kek')

from azure.identity import DefaultAzureCredential
from notebooks.envelope_encryption_v2.common import azure_crypto_functions as az_crypto

credential = DefaultAzureCredential()
key_vault_url = dbutils.widgets.get('key_vault_url')
key_name = dbutils.widgets.get('key_name')
az_crypto.create_keyvault_key(key_vault_url, key_name, credential)


## Create key table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS crypto.keyvault (
  key_alias STRING,
  key_version INT,
  key BINARY,
  vault_url STRING,
  key_name STRING,
  key_enabled BOOLEAN);


## Define encryption functions

In [0]:
%sql
USE SCHEMA crypto;
CREATE OR REPLACE FUNCTION crypto.unwrap_key(vault_url STRING, key_name STRING, encrypted_dek BINARY)
RETURNS BINARY
LANGUAGE PYTHON
PARAMETER STYLE SCALAR
HANDLER 'notebooks.envelope_encryption_v2.common.azure_crypto_functions.unwrap_data_key'
CREDENTIALS (`<uc_service_credential>` DEFAULT);

CREATE OR REPLACE FUNCTION crypto.encrypt(col STRING, catalog STRING, schema STRING)
RETURNS STRING
RETURN base64(aes_encrypt(col,
    (SELECT crypto.unwrap_key(vault_url, key_name, key) FROM crypto.keyvault 
     WHERE key_alias = CONCAT(catalog, '.', schema) AND key_enabled = true 
     ORDER BY key_version DESC LIMIT 1),
    'GCM','DEFAULT'));

CREATE OR REPLACE FUNCTION crypto.decrypt(col STRING, catalog STRING, schema STRING)
RETURNS STRING
RETURN CASE WHEN is_account_group_member(CONCAT(catalog, '.', schema, '.crypto.user')) THEN
  nvl(CAST(try_aes_decrypt(unbase64(col),
      (SELECT crypto.unwrap_key(vault_url, key_name, key) FROM crypto.keyvault 
       WHERE key_alias = CONCAT(catalog, '.', schema) AND key_enabled = true 
       ORDER BY key_version DESC LIMIT 1),
      'GCM','DEFAULT') AS STRING), col)
ELSE col END;
